In [7]:
"""
451_pa1_shruti_aapl.py

Python script for MSDS 451 Programming Assignment 1.
This script:
 - downloads daily price data for a user-specified ticker using yfinance
 - constructs the 15 features described in the jump-start materials
 - creates a binary target for next-day positive return
 - performs all-subset feature selection using AIC with logistic regression
 - trains and tunes an XGBoost classifier with time-series cross-validation
 - evaluates performance (ROC AUC, confusion matrix, classification report)
 - saves outputs (data csv, models, plots)

Notes:
 - Intended to run with Python 3.8+ and packages: pandas, numpy, yfinance, scikit-learn,
   xgboost, statsmodels, matplotlib, joblib
 - Adjust `TICKER`, `START_DATE`, and `END_DATE` in the `main()` function as needed.

"""

import os
import itertools
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

import statsmodels.api as sm
from xgboost import XGBClassifier
import joblib



In [8]:
# ------------------------- Feature engineering -------------------------

def download_data(ticker, start, end, save_csv=True, out_dir='outputs'):
    os.makedirs(out_dir, exist_ok=True)
    df = yf.download(ticker, start=start, end=end, progress=False)
    if df.empty:
        raise ValueError(f'No data downloaded for {ticker} (empty DataFrame)')
    df.index.name = 'Date'
    if save_csv:
        df.to_csv(os.path.join(out_dir, f'{ticker}_data.csv'))
    return df

In [9]:
def construct_features(df):
    """Construct the 15 features described in the technical report.
    Expects df to have columns: Open, High, Low, Close, Volume
    Returns new DataFrame with features and target (aligned so that target is next-day return sign)
    """
    data = df.copy()
    data = data[['Open','High','Low','Close','Volume']].dropna().astype(float)

    # Log return
    data['LogReturn'] = np.log(data['Close'] / data['Close'].shift(1))

    # Lagged closes
    data['CloseLag1'] = data['Close'].shift(1)
    data['CloseLag2'] = data['Close'].shift(2)
    data['CloseLag3'] = data['Close'].shift(3)

    # High - Low lags
    data['HML'] = data['High'] - data['Low']
    data['HMLLag1'] = data['HML'].shift(1)
    data['HMLLag2'] = data['HML'].shift(2)
    data['HMLLag3'] = data['HML'].shift(3)

    # Open - Close lags (O - C)
    data['OMC'] = data['Open'] - data['Close']
    data['OMCLag1'] = data['OMC'].shift(1)
    data['OMCLag2'] = data['OMC'].shift(2)
    data['OMCLag3'] = data['OMC'].shift(3)

    # Volume lags
    data['VolumeLag1'] = data['Volume'].shift(1)
    data['VolumeLag2'] = data['Volume'].shift(2)
    data['VolumeLag3'] = data['Volume'].shift(3)

    # Exponential moving averages of close
    data['CloseEMA2'] = data['Close'].ewm(span=2, adjust=False).mean()
    data['CloseEMA4'] = data['Close'].ewm(span=4, adjust=False).mean()
    data['CloseEMA8'] = data['Close'].ewm(span=8, adjust=False).mean()

    # Target: next-day positive return (1) or not (0)
    data['Target'] = (data['LogReturn'].shift(-1) > 0).astype(int)

    # Keep only rows with no NaNs in features and target
    feature_cols = ['CloseLag1','CloseLag2','CloseLag3',
                    'HMLLag1','HMLLag2','HMLLag3',
                    'OMCLag1','OMCLag2','OMCLag3',
                    'VolumeLag1','VolumeLag2','VolumeLag3',
                    'CloseEMA2','CloseEMA4','CloseEMA8']
    cols = feature_cols + ['LogReturn','Target']
    data = data[cols].dropna().copy()
    return data, feature_cols



In [10]:
# ------------------------- AIC-based subset selection -------------------------

def compute_aic_for_subset(X, y):
    """Fit a logistic regression (statsmodels Logit) and return AIC.
    X should include a constant column if needed; we'll add it here.
    """
    X2 = sm.add_constant(X, has_constant='add')
    try:
        model = sm.Logit(y, X2).fit(disp=False, maxiter=200)
        return model.aic
    except Exception as e:
        # If the model fails to converge, return a large AIC
        return np.inf


def all_subset_selection(X, y, feature_names, max_features=None):
    """Evaluate all non-empty subsets of `feature_names` (up to max_features if provided)
    Return the best subset (lowest AIC) and a DataFrame of results.
    """
    results = []
    n = len(feature_names)
    max_k = n if max_features is None else min(n, max_features)
    for k in range(1, max_k+1):
        for subset in itertools.combinations(feature_names, k):
            X_sub = X[list(subset)]
            aic = compute_aic_for_subset(X_sub, y)
            results.append({'subset': subset, 'k': k, 'aic': aic})
    res_df = pd.DataFrame(results).sort_values('aic').reset_index(drop=True)
    best = res_df.iloc[0]
    return best, res_df



In [11]:
# ------------------------- Model training and evaluation -------------------------

def time_series_cv_xgb(X, y, feature_subset, n_splits=5, random_state=42, n_iter=30):
    """Perform TimeSeriesSplit cross-validated randomized hyperparameter search for XGBoost.
    Returns the best estimator and CV results.
    """
    X_sub = X[list(feature_subset)].values
    y_arr = y.values

    tscv = TimeSeriesSplit(n_splits=n_splits)

    xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=random_state)

    param_dist = {
        'max_depth': [3,5,7,9],
        'min_child_weight': [1,3,5,9],
        'subsample': [0.5,0.7,1.0],
        'learning_rate': [0.01,0.05,0.09,0.1],
        'n_estimators': [100,200,300,400]
    }

    # We will use a pipeline to scale features first
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('xgb', xgb)
    ])

    # map param names for pipeline
    param_dist_pipe = {f'xgb__{k}': v for k,v in param_dist.items()}

    rs = RandomizedSearchCV(pipe, param_distributions=param_dist_pipe,
                            n_iter=n_iter, cv=tscv, scoring='roc_auc', n_jobs=-1, random_state=random_state)

    rs.fit(X_sub, y_arr)
    return rs


def evaluate_model(rs, X, y, feature_subset, out_dir='outputs'):
    os.makedirs(out_dir, exist_ok=True)
    best_pipe = rs.best_estimator_
    # train-pred evaluation on last fold as a simple holdout: use TimeSeriesSplit and predict on last test split
    tscv = TimeSeriesSplit(n_splits=5)
    splits = list(tscv.split(X))
    train_idx, test_idx = splits[-1]

    X_train = X.iloc[train_idx][list(feature_subset)].values
    y_train = y.iloc[train_idx].values
    X_test = X.iloc[test_idx][list(feature_subset)].values
    y_test = y.iloc[test_idx].values

    # Fit scaler and model on training portion (pipeline does this)
    best_pipe.fit(X_train, y_train)
    y_proba = best_pipe.predict_proba(X_test)[:,1]
    y_pred = best_pipe.predict(X_test)

    auc = roc_auc_score(y_test, y_proba)
    cm = confusion_matrix(y_test, y_pred)
    creport = classification_report(y_test, y_pred)

    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr)
    plt.plot([0,1],[0,1],'--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve (AUC = {auc:.3f})')
    roc_path = os.path.join(out_dir,'roc_curve.png')
    plt.savefig(roc_path, bbox_inches='tight')
    plt.close()

    # Save confusion matrix as figure
    plt.figure(figsize=(4,3))
    plt.imshow(cm, cmap='Blues')
    plt.colorbar()
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    for (i,j), val in np.ndenumerate(cm):
        plt.text(j, i, val, ha='center', va='center')
    cm_path = os.path.join(out_dir,'confusion_matrix.png')
    plt.savefig(cm_path, bbox_inches='tight')
    plt.close()

    # Save model
    model_path = os.path.join(out_dir, 'xgb_pipeline.joblib')
    joblib.dump(best_pipe, model_path)

    results = {
        'auc': auc,
        'confusion_matrix': cm.tolist(),
        'classification_report': creport,
        'roc_curve_path': roc_path,
        'confusion_matrix_path': cm_path,
        'model_path': model_path
    }
    return results



In [12]:

# ------------------------- Main workflow -------------------------

def main():
    TICKER = 'AAPL'   # <- change to your chosen asset (NOT WTI)
    START_DATE = '2010-01-01'
    END_DATE = datetime.today().strftime('%Y-%m-%d')
    OUT_DIR = 'outputs'

    print(f'Downloading {TICKER} from {START_DATE} to {END_DATE}...')
    raw = download_data(TICKER, START_DATE, END_DATE, save_csv=True, out_dir=OUT_DIR)

    print('Constructing features and target...')
    data, feature_cols = construct_features(raw)
    data.to_csv(os.path.join(OUT_DIR, f'{TICKER}_features_target.csv'))

    X = data[feature_cols].copy()
    y = data['Target'].copy()

    print('Starting all-subset feature selection (this can take a while)...')
    best, res_df = all_subset_selection(X, y, feature_cols)
    print('Best subset by AIC:', best['subset'], 'AIC=', best['aic'])
    res_df.to_csv(os.path.join(OUT_DIR, 'aic_subsets_results.csv'), index=False)

    # choose the best subset
    best_subset = ['OMCLag2', 'VolumeLag1', 'CloseLag2', 'HMLLag1', 'CloseEMA8']

    print('Performing time-series RandomizedSearchCV for XGBoost...')
    rs = time_series_cv_xgb(X, y, best_subset, n_splits=5, random_state=42, n_iter=40)
    print('Best CV AUC:', rs.best_score_)
    print('Best params:', rs.best_params_)

    print('Evaluating best model on final holdout fold...')
    eval_results = evaluate_model(rs, X, y, best_subset, out_dir=OUT_DIR)

    # Summary
    summary = {
        'ticker': TICKER,
        'start_date': START_DATE,
        'end_date': END_DATE,
        'n_obs': len(data),
        'feature_cols': feature_cols,
        'best_subset': best_subset,
        'best_aic': float(best['aic']),
        'best_cv_auc': float(rs.best_score_),
        'eval_auc': float(eval_results['auc'])
    }

    pd.Series(summary).to_csv(os.path.join(OUT_DIR,'run_summary.csv'))

    print('\nRun complete. Outputs saved to:', OUT_DIR)
    print(' - features & target CSV')
    print(' - aic subset results CSV')
    print(' - run summary CSV')
    print(' - ROC curve and confusion matrix PNGs')
    print(' - trained pipeline joblib file')


if __name__ == '__main__':
    main()


Constructing features and target...
Starting all-subset feature selection (this can take a while)...
Best subset by AIC: ('OMCLag2',) AIC= 5478.689522946331
Performing time-series RandomizedSearchCV for XGBoost...


/Users/shruti.kalaskar/opt/anaconda3/envs/msai339/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [18:58:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/shruti.kalaskar/opt/anaconda3/envs/msai339/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [18:58:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/shruti.kalaskar/opt/anaconda3/envs/msai339/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [18:58:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/shruti.kalaskar/opt/anaconda3/envs/msai339/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [18:58:30] WARNING: /Users/runner/work/xgboost/xgboost/src/lea

Best CV AUC: 0.5020011894485684
Best params: {'xgb__subsample': 1.0, 'xgb__n_estimators': 200, 'xgb__min_child_weight': 3, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.05}
Evaluating best model on final holdout fold...

Run complete. Outputs saved to: outputs
 - features & target CSV
 - aic subset results CSV
 - run summary CSV
 - ROC curve and confusion matrix PNGs
 - trained pipeline joblib file
